# The PatentsView Search API Tutorial
### For detailed information on using the PatentsView API, see:  
* [PatentsView API information page](https://patentsview.org/apis/api-endpoints/patentsbeta)  
* [PatentsView API Query Language](https://patentsview.org/apis/api-query-language)  
* [Swagger API Documentation and test page](https://search.patentsview.org/swagger-ui/)

In [1]:
import pandas as pd
import json
import requests

An API key is required to view results from the PatentsView API.  
An API key can be requested through the PatentsView Service Desk:  
https://patentsview-support.atlassian.net/servicedesk/customer/portals

In [2]:
api_key = "my_API_key"

Responses from the Patentsview API are formatted as JSON and always contain the key **"error"** .  
Successful responses will also contain the keys:  
  
**"count"** _(the number of records included in the response)_,  
  
**"total_hits"** _(the total number of records in the PatentsView data matching the parameters of the request)_,  
  
and another key containing the actual record data which is specific to each endpoint.  
The function below converts the endpoint to the expected results key.

In [3]:
special_keys = {
    'ipc' : 'ipcr',
    'wipo': 'wipo'
}

def response_key(endpoint:str) -> str:
    """
    Given the API endpoint contacted, returns the json key for the data returned by the API.
    In most cases, this is the plural of the term at the end of the endpoint URL.
    """
    endpoint = endpoint.rstrip('/')
    leaf = endpoint.split('/')[-1]
    if leaf in special_keys:
        return special_keys[leaf]
    elif leaf.endswith('s'):
        return leaf + 'es'
    else:
        return leaf + 's'

### Identify and set your endpoint of interest 

In [4]:
base_url = 'https://search.patentsview.org' # this is the same for all endpoints

available endpoints:  

| Endpoint Name | Endpoint Extension | single-ID GET? |
| ---: | :--- | :--- |
| Assignee | api/v1/assignee | assignee_id |
| Attorney | api/v1/patent/attorney | attorney_id |
| Inventor | api/v1/inventor | inventor_id |
| Location | api/v1/location | location_id |
| CPC Class | api/v1/cpc_class | cpc_class |
| CPC Subclass | api/v1/cpc_subclass | cpc_subclass |
| CPC Group | api/v1/cpc_group | cpc_group |
| IPC | api/v1/ipc | No |
| USPC Mainclass | api/v1/uspc_mainclass | uspc_mainclass_id |
| USPC Subclass | api/v1/uspc_subclass | uspc_subclass_id |
| WIPO | api/v1/wipo | wipo_id |
| Patent | api/v1/patent | patent_id |
| Foreign Citation | api/v1/patent/foreign_citation | No |
| Other Reference | api/v1/patent/otherreference | No |
| Related Application Text | api/v1/patent/rel_app_text | No |
| US Application Citation | api/v1/patent/us_application_citation | patent_id |
| US Patent Citation | api/v1/patent/us_patent_citation | patent_id |
  
See [Swagger API Documentation and test page](https://search.patentsview.org/swagger-ui/) for full details  

In [5]:
# get the endpoint that matches your query from the Swagger page or the list above
endpoint = 'api/v1/patent' # e.g.

### constructing your query:  
Parameters:
| param | name | object type | default |  
| ---: | :--- | :--- | :--- |  
| q | query | dict | None (required parameter) |  
| f | fields | list | differs by endpoint; see [Swagger page](https://search.patentsview.org/swagger-ui/) |  
| o |  options | dict | "size"\*: 100, "after": None (optional) |  
| s | sort | list of dict | differs by endpoint; see [Swagger page](https://search.patentsview.org/swagger-ui/) |  
  
See more detail on query syntax at the [PatentsView API Query Language](https://patentsview.org/apis/api-query-language) Page  
  
*\*Note: the maximum number of results returned for a single query is 1000. larger values for "size" will be accepted, but only 1000 results will be returned*

Some endpoints contain groups of fields representing related entities connected to one of that endpiont's primary entity type; for example, the patent endpoint contains a field "inventors", which contains information on all inventors associated with any given patent. The fields for related entities can be requested in the API request's fields parameter as a group by using the group name in the fields parameter, or individually by specifying the required field as `"{entity_type}.{subfield}"`.  
  
Because there are often many of these related entities connected to a single central entity, the details for related entities are packaged as a list of JSON objects within the full response JSON content.  
  
For details on unpacking these responses and warnings on how these fields behave in queries, see the below [Appendix on related entity queries](#Appendix-on-related-entity-queries)

See the [Swagger API Documentation and test page](https://search.patentsview.org/swagger-ui/) for the full list of available fields for each endpoint.  

In [6]:
# any of these except q can be omitted
param_dict = {
    "f" : ["patent_id", "patent_title", "patent_date"],
    "o" : {"size":200},
    "q" : {"patent_id":["D345393","10905426","11172927"]},
    "s" : [{"patent_id":"asc"}], 
}

### GET requests:  
parameters provided as part of the url after the endpoint, separated by "&"

In [7]:
param_string = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in param_dict.items()])
# note the json.dumps to ensure strings are surrounded by double quotes instead of single
# the api will not accept single quotes
query_url = f"{base_url}/{endpoint.strip('/')}/?{param_string}"

In [8]:
response = requests.get(query_url, headers={"X-Api-Key": api_key})
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             3
total_hits        3
dtype: object


In [9]:
response_unpacked = pd.DataFrame(response.json()[response_key(endpoint)])
display(response_unpacked.head())

,patent_id,patent_title,patent_date
0,10905426,Detachable motor powered surgical instrument,2021-02-02
1,11172927,Staple cartridges for forming staples having d...,2021-11-16
2,D345393,Turtle toy car,1994-03-22


### POST requests:  
parameters provided inside a json request body

In [10]:
query_url = f"{base_url}/{endpoint.strip('/')}"
headers = {
    "X-Api-Key": api_key,
    "accept": "application/json",
    "Content-Type": "application/json"
}

In [11]:
response = requests.post(url=query_url, headers=headers, json=param_dict)
# note that the POST request takes the parameter dictionary directly rather than requiring json as text
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             3
total_hits        3
dtype: object


In [12]:
response_unpacked = pd.DataFrame(response.json()[response_key(endpoint)])
display(response_unpacked.head())

,patent_id,patent_title,patent_date
0,10905426,Detachable motor powered surgical instrument,2021-02-02
1,11172927,Staple cartridges for forming staples having d...,2021-11-16
2,D345393,Turtle toy car,1994-03-22


### Single Record GET Requests  
Many of the endpoints also allow providing a single id at the end of the endpoint URL to retrieve default information about that record

In [13]:
record_id = "D345393" # e.g. a patent ID for the patents endpoint

In [14]:
query_url = f"{base_url}/{endpoint.strip('/')}/{record_id}/"
response = requests.get(query_url, headers={"X-Api-Key": api_key})
print(response)
if response.status_code == 200:
    print(pd.Series(response.json())[:-1])
else: 
    print(pd.Series(response.headers)) 

<Response [200]>
error         False
count             1
total_hits        1
dtype: object


In [15]:
display(pd.Series(response.json()[response_key(endpoint)][0]))

patent_id              D345393
patent_title    Turtle toy car
patent_date         1994-03-22
dtype: object

---
# Appendix on related entity queries
### Identifying related entity fields in Swagger
Related entity groups can be identified in the Swagger Documentation page by this structure:  
(the applicants group in the patents endpoint given as an example)  
  
In the endpoint details panels:  
![the applicants group defined in the patents endpoint details panel](notebook_images/detail_ex.png)  
In the response schema panels:  
![the applicants group defined in the patents endpoint response schema panel](notebook_images/schema_ex.png)  

### unpacking related entity API results
Related entity records can be expanded and parsed with the Pandas DataFrame methods `explode()` and `apply()`:

In [16]:
# an example case: 3 patents from 2021, with inventors
example_endpoint = "api/v1/patent"

example_params = { 
    "f" : ["patent_id","patent_title", "patent_date", "inventors"],
    "o" : {"size":3},
    "q" : {"patent_year":"2021"},
}

example_query = "&".join([f"{param_name}={json.dumps(param_val)}" for param_name, param_val in example_params.items()])
example_url = f"{base_url}/{example_endpoint.strip('/')}/?{example_query}"

response = requests.get(example_url, headers={"X-Api-Key": api_key})

example_data = pd.DataFrame(response.json()[response_key(example_endpoint)])
display(example_data)

,patent_id,patent_title,patent_date,inventors
0,10881042,Agricultural implement with a scraper internal...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...
2,10881044,Planter with full tandem offset pivot,2021-01-05,[{'inventor': 'https://search.patentsview.org/...


In [17]:
# use explode() to get one record for each inventor:
example_exploded = example_data.explode('inventors')
display(example_exploded)

,patent_id,patent_title,patent_date,inventors
0,10881042,Agricultural implement with a scraper internal...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
1,10881043,Method for increasing the load on driving rear...,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
2,10881044,Planter with full tandem offset pivot,2021-01-05,{'inventor': 'https://search.patentsview.org/a...
2,10881044,Planter with full tandem offset pivot,2021-01-05,{'inventor': 'https://search.patentsview.org/a...


In [18]:
# extract individual inventor fields by applying pd.Series to the exploded inventor JSON:
example_inventors = example_exploded['inventors'].apply(pd.Series)
display(example_inventors)

,inventor,inventor_name_first,inventor_name_last,inventor_city,inventor_state,inventor_country,inventor_sequence
0,https://search.patentsview.org/api/v1/inventor...,Travis Erlin,Westlind,Langdon,ND,US,0
1,https://search.patentsview.org/api/v1/inventor...,Marek,Zak,Brno,None,CZ,3
1,https://search.patentsview.org/api/v1/inventor...,Frantisek,Bauer,Brno,None,CZ,0
1,https://search.patentsview.org/api/v1/inventor...,Martin,Fajman,Brno,None,CZ,2
1,https://search.patentsview.org/api/v1/inventor...,Adam,Polcar,Polička,None,CZ,4
1,https://search.patentsview.org/api/v1/inventor...,Jiri,Cupera,Brno,None,CZ,1
2,https://search.patentsview.org/api/v1/inventor...,Monte J.,Rans,Hesston,KS,US,1
2,https://search.patentsview.org/api/v1/inventor...,Benjamin Anson,Fanshier,Hesston,KS,US,0


In [19]:
# re-connect the expanded inventor data to the original patent data by their shared index:
example_recombined = example_data.merge(example_inventors, left_index=True, right_index=True)

In [20]:
# or, combining all three steps:
example_recombined = example_data.merge(example_data['inventors'].explode().apply(pd.Series), 
                                        left_index=True, right_index=True)

In [21]:
display(example_recombined)

,patent_id,patent_title,patent_date,inventors,inventor,inventor_name_first,inventor_name_last,inventor_city,inventor_state,inventor_country,inventor_sequence
0,10881042,Agricultural implement with a scraper internal...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Travis Erlin,Westlind,Langdon,ND,US,0
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Marek,Zak,Brno,None,CZ,3
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Frantisek,Bauer,Brno,None,CZ,0
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Martin,Fajman,Brno,None,CZ,2
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Adam,Polcar,Polička,None,CZ,4
1,10881043,Method for increasing the load on driving rear...,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Jiri,Cupera,Brno,None,CZ,1
2,10881044,Planter with full tandem offset pivot,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Monte J.,Rans,Hesston,KS,US,1
2,10881044,Planter with full tandem offset pivot,2021-01-05,[{'inventor': 'https://search.patentsview.org/...,https://search.patentsview.org/api/v1/inventor...,Benjamin Anson,Fanshier,Hesston,KS,US,0


### Queries using related entity fields  
Related entity subfields can also be used in query criteria, specified in the `"{superfield}.{subfield}"` form. It should be noted, however, that queries always apply conditions on the scope of the central entity of the endpoint. There are two consequences of this behavior that are important to understand:
1. **The related entities returned will not be filtered by the criteria applied in the query.**  

For example, using the query `"q":{"inventors.inventor_country":"UK"}` on the endpoint `api/v1/patent` will return only and all patents where *at least one* inventor is located in the UK, but will provide all inventors for each returned patent packaged within the "inventors" response field, regardless of each individual inventor's location.

2. **When applying multiple conditions to related-entity fields, a central entity record will be returned if any combination of its related entities satisfy those conditions.**  

In order to retrieve records for which individual related entities satisfy a particular combination of criteria (for example, finding patents with a specific inventor identified by first and last name), you must first search for that related entity within its own endpoint to retrieve its entity ID, and then use that ID in the search criteria for the central entity.

As a concrete example, to search for patents invented by George Washington and Abraham Lincoln, it would not be sufficient to search `api/v1/patent` with the query 
```
"q": {"_or":[{"_and":[{"inventors.inventor_first_name":"George"}, {"inventors.inventor_last_name":"Washington"}]},
             {"_and":[{"inventors.inventor_first_name":"Abraham"}, {"inventors.inventor_last_name":"Lincoln"}]}]}
```
as this would include results with combinations of inventors such as George Smith and Jane Washington, or Abraham Johnson and Sarah Lincoln, since each of these combinations would satisfy one of the necessary combinations of names within the same patent.  
Instead, you would first need to search `api/v1/inventor` with the query
```
"q": {"_or":[{"_and":[{"inventor_first_name":"George"}, {"inventor_last_name":"Washington"}]},
             {"_and":[{"inventor_first_name":"Abraham"}, {"inventor_last_name":"Lincoln"}]}]}
```
and note the returned `inventor_id` values. If this query returned results with the `inventor_id` values "GWash1" and "ALinc1", you could then search `api/v1/patent` with the query
```
"q": {"inventors.inventor_id":["GWash1","ALinc1"]}
```
which would return only patents with one of those two `inventor_id`s among their inventors